# **Libraries import**

In [3]:
import numpy as np
from skimage import (exposure, filters, measure, morphology, util)
import tifffile as tif
from pathlib import Path
from PIL import Image
import napari

# **Image preparation**

In [4]:
class Image_preparation():
    
    def __init__(self):
        self.image_path = image_path
        
    def reading_image(self):
        image = Image.open(image_path)
        image = np.array(image)
        return image
    
    def info_image(image):
        print("Image shape: {}".format(image.shape))
        print("Image data type: {}".format(image.dtype))
        print("Image range: ({}, {})".format(np.min(image), np.max(image)))
        
    def show_plane(ax, plane, cmap="gray", title=None):
        ax.imshow(plane, cmap=cmap)
        ax.axis("off")

        if title:
            ax.set_title(title)
        
    def show_image(image):
        (n_plane, n_row, n_col) = image.shape
        _, (a, b, c) = plt.subplots(ncols=3, figsize=(15, 5))
        show_plane(a, image[n_plane // 2], title=f'Plane = {n_plane // 2}')
        show_plane(b, image[:, n_row // 2, :], title=f'Row = {n_row // 2}')
        show_plane(c, image[:, :, n_col // 2], title=f'Column = {n_col // 2}')

# **2D segmentation**

In [6]:
class Segmentation_2D():
    
    def __init__(self):
        self.image = image
    
    def from_3D_to_2D(image):
        image_2D = image[n_plane // 2]
        return image_2D
        
    def image_processing(image_2D, sigma):
        
        # Applying the adapted histogram equalization 
        image_ahe = exposure.equalize_adapthist(image_2D) 
        
        # Deleting extreme pixel values to avoid salt and pepper noise
        vmin, vmax = np.quantile(image_ahe, q=(0.05, 0.95))
        stretched = exposure.rescale_intensity(
            image_ahe, 
            in_range=(vmin, vmax), 
            out_range=np.float32) 
        
        # Applying a gaussian filter with sigma = 4
        filtered_image = skimage.filters.gaussian(stretched, sigma = sigma)
        return filtered_image
        
    def segmentation_2D(filtered_image):       
        # Applying the Otsu threshold method
        thresh = skimage.filters.threshold_otsu(filtered_image)
        binary = filtered_image > thresh 
        return binary
        
    def cleaning(binary, width):
        filled = ndi.binary_fill_holes(binary)
        opened = ndi.binary_opening(filled)
        holes = morphology.remove_small_holes(opened, area_threshold = width * 10)
        result = morphology.remove_small_objects(holes, min_size = width * 30, connectivity = 100)
        return result
        
    def show_result(image_2D, result):
        _, (a, b) = plt.subplots(ncols=2, figsize=(15, 5))
        show_plane(a, image_2D, title=f'Original')
        show_plane(b, result, title=f'Segmented')

# **3D segmentation**

In [ ]:
class Segmentation_3D():
    
    def __init__(self):
        self.image = image
        
    def image_processing(image):
        
        # Applying the adapted histogram equalization 
        image_ahe_3D = exposure.equalize_adapthist(image) 
        
        # Deleting extreme pixel values to avoid salt and pepper noise
        vmin, vmax = np.quantile(image_ahe, q=(0.05, 0.95))
        stretched_3D = exposure.rescale_intensity(
            image_ahe_3D, 
            in_range=(vmin, vmax), 
            out_range=np.float32)
        
        # Applying a gaussian filter with sigma = 4
        filtered_image_3D = skimage.filters.gaussian(stretched_3D, sigma = 2)
        return filtered_image_3D
    
    def segmentation_3D(filtered_image_3D):
        # Applying the Otsu threshold method
        thresh = skimage.filters.threshold_otsu(filtered_image_3D)
        binary = filtered_image_3D > thresh 
        return binary
    
    def cleaning(binary, width):
        filled = ndi.binary_fill_holes(binary)
        opened = ndi.binary_opening(filled)
        holes = morphology.remove_small_holes(opened, area_threshold = width * 10)
        result = morphology.remove_small_objects(holes, min_size = width * 30, connectivity = 100)
        return result
        
    def show_result(image, result):
        viewer = napari.view_image(image)
        viewer.add_image(result)

# **Measurements**

In [ ]:
class Measurements():
    
    def __init__(self):
        self.result = result
    
    def regions_labeling(self):
        image = util.img_as_ubyte(result) > 110
        labels_image = measure.label(result, connectivity=image.ndim)
        regions = measure.regionprops(labels_image)
        return regions
    
    def get_regions_info(regions, spacing_row, spacing_col):
        info_table = pd.DataFrame(measure.regionprops_table(label_img, properties=['area', 'major_axis_length', 'minor_axis_length']))
        per_pixel_area = spacing_row * spacing_col
        num_white_pixels = np.nonzero(result)
        info_table['area (mm²)'] = info_table['area'] * per_pixel_area
        return info_table